[Home](../../README.md)

### Seoul Bike Sharing

**Dataset:** https://archive.ics.uci.edu/dataset/560/seoul+bike+sharing+demand

We will wrangle the bike data set previewed in the last [Jupyter Notebook](2.model_development/2.1.data_wrangling/2.1.1.data_preview.ipynb).

> None of these processes are destructive to the source CSV as long as you save the modified data to a new CSV.

#### Load the required dependencies

In [1]:
# Import frameworks
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

#### Store the data as a local variable

The data frame is a Pandas object that structures your tabular data into an appropriate format. It loads the complete data in memory so it is now ready for preprocessing. It integrates seamlessly with other data science and machine learning libraries in Python, such as NumPy, SciPy, scikit-learn, and Matplotlib. Additionaly, using a DataFrame makes our code more readble and maintainable.

In [2]:
data_frame = pd.read_csv("2.1.2.SeoulBikeData_Sample_Data.csv")

#### Dealing with null values

Null values during data analysis can cause runtime errors and unexpected results. It is important to identify null values and deal with them appropriately before training a model.

The `isnull().sum()` method call returns the null values in any column.

- Null data was not captured due to the dataset coming from a reputable source: https://archive.ics.uci.edu

    - Additionally, mentioned in the source description

In [3]:
data_frame.isnull().sum()

Date              0
Count             0
Hour              0
Temp              0
Humidity          0
WindSpeed         0
Visibility        0
DewPointTemp      0
SolarRadiation    0
Rainfall          0
Snowfall          0
Seasons           0
Holiday           0
FunctioningDay    0
dtype: int64

* If we were to detect null values we could remove any row with a null value with a `dropna()` method call.

In [4]:
data_frame = data_frame.dropna(subset=['Seasons'])
data_frame.isnull().sum()

Date              0
Count             0
Hour              0
Temp              0
Humidity          0
WindSpeed         0
Visibility        0
DewPointTemp      0
SolarRadiation    0
Rainfall          0
Snowfall          0
Seasons           0
Holiday           0
FunctioningDay    0
dtype: int64

#### Remove duplicates

Duplicate data can have detrimental effects on your machine learning models and outcomes, such as reducing data diversity and representativeness, which can lead to overfitting or biased models.

The `duplicated().sum()` method call returns the count of duplicate rows in the data frame.

- No duplicates were detected

In [4]:
data_frame.duplicated().sum()

np.int64(0)

* If we were to find duplicates we could use the drop_duplicates() method call, then stored back onto the data_frame variable removing the duplicates.

In [20]:
data_frame = data_frame.drop_duplicates()
data_frame.duplicated().sum()

np.int64(0)

#### Replace data

We can run a lambda function on a column to modify its values. For a simple example, let’s convert the Seasons to lowercase. To run a function over a complete column, we can use the apply method which iterates over each row and modifies the values.

* Replaced upper cases for seasons for better access, **will later be overwritten by binary**

In [5]:
data_frame['Seasons'] = data_frame['Seasons'].apply(lambda x: x.lower())
data_frame['Seasons'].head()

0    winter
1    winter
2    winter
3    winter
4    winter
Name: Seasons, dtype: object

We can check that there are no data entry errors by the `unique()` method call.

-  No data entry errors were caught.

In [6]:
data_frame['Seasons'].unique()

array(['winter', 'spring', 'summer', 'autumn'], dtype=object)

In [7]:
data_frame['Seasons'] = data_frame['Seasons'].apply(lambda season: 'winter' if season.lower() == 'winter' else ('summer' if season.lower() == 'summer' else season))
data_frame['Seasons'].unique()

array(['winter', 'spring', 'summer', 'autumn'], dtype=object)

#### Remove outliers

Outliers can skew your analysis on numerical columns, and it is important to remove them. We can use the 25th and 75th quartile on numerical data, to get the inter-quartile range. This allows us to estimate an acceptable range, and we can then filter out any values outside this range. Mathematically, outliers are values occurring outside 1.5 times the interquartile range (IQR) from the first quartile (Q1) or third quartile (Q3).

In [8]:
# Get the inter-quartile range
print(data_frame['Count'].describe())
Q1 = data_frame['Count'].quantile(0.25)
Q3 = data_frame['Count'].quantile(0.75)
IQR = Q3 - Q1
print(f'Outliers are a Count above {Q3 + IQR * 1.5} or below {Q1 - IQR * 1.5}')

count    8760.000000
mean      704.602055
std       644.997468
min         0.000000
25%       191.000000
50%       504.500000
75%      1065.250000
max      3556.000000
Name: Count, dtype: float64
Outliers are a Count above 2376.625 or below -1120.375


In [9]:
data_frame = data_frame[(data_frame['Count'] >= Q1 - 1.5 * IQR) & (data_frame['Count'] <= Q3 + 1.5 * IQR)]
print(data_frame['Count'].describe())

count    8602.000000
mean      667.971635
std       589.552620
min         0.000000
25%       188.000000
50%       485.000000
75%      1030.750000
max      2375.000000
Name: Count, dtype: float64


#### Scaling features to a common range

Scaling the features makes it easier for machine learning algorithms to find the optimal solution, as the different scales of the features do not influence them.

* I left room for outliers with my `max_count = 2400`

In [8]:
scale_feature = 'Count'

#the minimum value with space for outliers
MIN_COUNT = 0

#the maximum value with space for outliers
MAX_COUNT = 2400

#scale features
data_frame[scale_feature] = [(X - MIN_COUNT) / (MAX_COUNT - MIN_COUNT) for X in data_frame[scale_feature]]

data_frame.describe()

,Count,Hour,Temp,Humidity,WindSpeed,Visibility,DewPointTemp,SolarRadiation,Rainfall,Snowfall
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000
mean,0.293584,11.500000,12.882922,58.226256,1.724909,1436.825799,4.073813,0.569111,0.148687,0.075068
std,0.268749,6.922582,11.944825,20.362413,1.036300,608.298712,13.060369,0.868746,1.128193,0.436746
min,0.000000,0.000000,-17.800000,0.000000,0.000000,27.000000,-30.600000,0.000000,0.000000,0.000000
25%,0.079583,5.750000,3.500000,42.000000,0.900000,940.000000,-4.700000,0.000000,0.000000,0.000000
50%,0.210208,11.500000,13.700000,57.000000,1.500000,1698.000000,5.100000,0.010000,0.000000,0.000000
75%,0.443854,17.250000,22.500000,74.000000,2.300000,2000.000000,14.800000,0.930000,0.000000,0.000000
max,1.481667,23.000000,39.400000,98.000000,7.400000,2000.000000,27.200000,3.520000,35.000000,8.800000


> [!important]
> You need to save the calculations for each dataset you scale for scaling new values for prediction.
> - Calculations were saved for scaling new values for prediction.

* I also counted how many 0 values there were, due to issues with linear regression (later evident)

In [11]:
zero_counts = data_frame.isin([0]).sum()
print("Number of 0 values in each column:")
print(zero_counts)

Number of 0 values in each column:
Date                 0
Count              295
Hour               365
Temp                21
Humidity            17
WindSpeed           74
Visibility           0
DewPointTemp        59
SolarRadiation    4294
Rainfall          8075
Snowfall          8159
Seasons              0
Holiday              0
FunctioningDay       0
dtype: int64


* Rainfall, snowfall and solar radiation appear to have a lot, however these values are in decimals and have little effect

* Having 300 Count values as 0 can possibly have issues. However, this may be maintanence days etc.

    * Removing these values also decreased test scores by 0.3

#### Save the wrangled data to CSV

In [9]:
data_frame.to_csv('../2.2.feature_engineering/2.2.1.wrangled_data.csv', index=False)